In [4]:
import pandas as pd
import os
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact
import plotly.graph_objects as go
import ruptures as rpt
from itertools import combinations as comb
from statsmodels.stats import power
import numpy as np

from IPython.display import display
pd.set_option('display.max_rows', None)
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from scipy.spatial.distance import cdist
import colorsys

In [34]:
folder_path = 'data'
def read_data(filename):
    accelerometer = pd.read_csv(os.path.join(folder_path, filename, 'Accelerometer.csv'),sep=';')
    accelerometer['Time (s)'] = pd.to_datetime(accelerometer['Time (s)'], unit='s')
    accelerometer =accelerometer.set_index('Time (s)')
    accelerometer = accelerometer.resample('2.5ms').mean()
    accelerometer.rename(columns={'Acceleration x (m/s^2)':'accelerometer_x','Acceleration y (m/s^2)':'accelerometer_y','Acceleration z (m/s^2)':'accelerometer_z'}, inplace=True)
    accelerometer =accelerometer[9000:-9000]
    accelerometer.reset_index(inplace=True)

    gyroscope = pd.read_csv(os.path.join(folder_path, filename,'Gyroscope.csv'),sep=';')
    gyroscope['Time (s)'] = pd.to_datetime(gyroscope['Time (s)'], unit='s')
    gyroscope =gyroscope.set_index('Time (s)')
    gyroscope = gyroscope.resample('2.5ms').mean()
    gyroscope.rename(columns={'Gyroscope x (rad/s)':'gyroscope_x','Gyroscope y (rad/s)':'gyroscope_y','Gyroscope z (rad/s)':'gyroscope_z'}, inplace=True)
    gyroscope = gyroscope[9000:-9000]
    gyroscope.reset_index(inplace=True)
    merged = pd.merge(accelerometer,gyroscope, on= 'Time (s)', how='inner')
    return merged

In [35]:
lucas1 = read_data('lucas1')
nick1 = read_data('nick1')
till1 = read_data('till1')
till2 = read_data('till2')
lusia1 = read_data('luisa1')
nelel1 = read_data('nele1')
katarina1 = read_data('katarina1')
uta1 = read_data('uta1')
uta2 = read_data('uta2')
nick3 = read_data('nick3')

In [36]:
nick1['label'] = 0
till1['label'] = 1
till2['label'] = 1
lucas1['label'] = 2
lusia1['label'] = 3
katarina1['label'] = 4
nelel1['label'] = 5
uta1['label'] = 6
uta2['label'] = 6
nick3['label'] = 0

In [37]:
def create_sliding_windows(data, window_length, stride):
    # Extract the feature columns and class labels
    features = data.iloc[:, 1:-1].values  # Exclude the first column and the last column
    labels = data.iloc[:, -1].values      # The last column is the class label

    # Number of windows that can be created from the data with the given stride
    num_windows = (len(data) - window_length) // stride + 1

    # Create sliding windows using numpy stride tricks
    windows = np.lib.stride_tricks.sliding_window_view(features, (window_length, features.shape[1]))
    windows = windows[::stride, 0, :, :]  # Apply stride to the windows
    y_windows = labels[np.arange(0, num_windows * stride, stride)]  # Class labels at the start of each window

    return windows, y_windows

window_length = 100
stride = 100

dataframes = [nick1,nick3,till1,lucas1,lusia1,katarina1,nelel1,uta1,uta2] 

X_train_list = []
y_train_list = []
X_test_list = []
y_test_list = []

for df in dataframes:
    X_windows, y_windows = create_sliding_windows(df, window_length, stride)
    X_train, X_test = X_windows[:int(0.8 * len(X_windows))], X_windows[int(0.8 * len(X_windows)):]
    y_train, y_test = y_windows[:int(0.8 * len(y_windows))], y_windows[int(0.8 * len(y_windows)):]
    
    X_train_list.append(X_train)
    y_train_list.append(y_train)
    X_test_list.append(X_test)
    y_test_list.append(y_test)

X_train = np.concatenate(X_train_list, axis=0)
y_train = np.concatenate(y_train_list, axis=0)
X_test = np.concatenate(X_test_list, axis=0)
y_test = np.concatenate(y_test_list, axis=0)

In [38]:
n_samples, window_length, n_features = X_train.shape
X_train_reshaped = X_train.reshape(n_samples, window_length * n_features)
X_test_reshaped = X_test.reshape(X_test.shape[0], window_length * n_features)

model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model.fit(X_train_reshaped, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [39]:
y_pred = model.predict(X_test_reshaped)
y_pred_proba = model.predict_proba(X_test_reshaped)


accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print("X_train shape:", X_train_reshaped.shape)
print("X_test shape:", X_test_reshaped.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

Accuracy: 0.8633808906182447
X_train shape: (18484, 6)
X_test shape: (4626, 6)
y_train shape: (18484,)
y_test shape: (4626,)


In [40]:
report = classification_report(y_test, y_pred)

print(report)

              precision    recall  f1-score   support

           0       0.90      0.94      0.92      1382
           1       0.81      0.77      0.79       533
           2       0.98      0.93      0.95       319
           3       0.87      0.79      0.83       511
           4       0.81      0.88      0.85       372
           5       0.82      0.83      0.82       398
           6       0.85      0.84      0.84      1111

    accuracy                           0.86      4626
   macro avg       0.86      0.85      0.86      4626
weighted avg       0.86      0.86      0.86      4626



In [27]:
report = classification_report(y_test, y_pred)

print(report)

              precision    recall  f1-score   support

           0       0.87      0.90      0.89       697
           1       0.83      0.76      0.79       533
           2       0.98      0.94      0.96       319
           3       0.86      0.82      0.84       511
           4       0.79      0.91      0.85       372
           5       0.82      0.82      0.82       398
           6       0.86      0.85      0.85      1111

    accuracy                           0.85      3941
   macro avg       0.86      0.86      0.86      3941
weighted avg       0.85      0.85      0.85      3941



In [12]:
y_pred_proba.min(), y_pred_proba.max()

(7.228297e-07, 0.99998057)

Verification with Till2

In [13]:
X_verification, y_verification = create_sliding_windows(till2, window_length, stride)
n_samples, window_length, n_features = X_verification.shape
X_verification = X_verification.reshape(n_samples, window_length * n_features)

In [14]:
y_pred_ver = model.predict(X_verification)
y_pred_proba_ver = model.predict_proba(X_verification)
accuracy = accuracy_score(y_pred_ver, y_verification)

In [15]:
y_pred_proba_ver.min(), y_pred_proba_ver.max()

(9.742447e-07, 0.9999852)

In [16]:
accuracy

0.9890368050117463

In [17]:
# def check_leakage(X_train, X_test, tol=1e-100):
#     # Check if any instance in the test set is present in the training set
#     for test_instance in X_test:
#         dists = np.linalg.norm(X_train - test_instance, axis=1)
#         if dists.min() < tol:
#             return True

#     # For time series data, check if any subsequence of the test set is present in the training set
#     for test_instance in X_test:
#         for i in range(X_train.shape[0] - len(test_instance) + 1):
#             if np.allclose(X_train[i:i+len(test_instance)], test_instance, atol=tol):
#                 return True

#     return False

# leakage_exists = check_leakage(X_train_reshaped, X_test_reshaped)
# print(f"Leakage exists: {leakage_exists}")

In [18]:
import plotly.graph_objects as go
import numpy as np
import colorsys

unique_classes = np.unique(y_train)
base_colors = ['rgb(228,26,28)', 'rgb(55,126,184)', 'rgb(77,175,74)', 'rgb(152,78,163)', 'rgb(255,127,0)']  

fig = go.Figure()

# Create a trace for each class and dimension
for j, cls in enumerate(unique_classes):
    indices = np.where(y_train == cls)[0]
    idx = np.random.choice(indices)
    window = X_train[idx]
    for i in range(6):
        rgb = base_colors[j].lstrip('rgb(').rstrip(')').split(',')
        hls = colorsys.rgb_to_hls(int(rgb[0])/255, int(rgb[1])/255, int(rgb[2])/255)
        hls = (hls[0], hls[1]*(1 - 0.1*i), hls[2])  # Adjust lightness
        rgb = colorsys.hls_to_rgb(*hls)
        color = f'rgb({int(rgb[0]*255)},{int(rgb[1]*255)},{int(rgb[2]*255)})'
        fig.add_trace(go.Scatter(y=window[:, i], mode='lines', name=f'Class {cls} Dimension {i+1}', line=dict(color=color)))

# Create buttons for each dimension
buttons = []
for i in range(6):
    buttons.append(dict(
        label=f"Dimension {i+1}",
        method="update",
        args=[{"visible": [j%6 == i for j in range(len(unique_classes)*6)]}]
    ))

# Add dropdown
fig.update_layout(
    updatemenus=[
        go.layout.Updatemenu(
            buttons=buttons,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.1,
            yanchor="top"
        ),
    ]
)

fig.show()